In [ ]:
import os
import pickle
import shapely
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

# Add main repo to path
import sys
from os.path import expanduser
sys.path.append(expanduser('../../'))

from spicy_snow.IO.user_dates import get_input_dates

from spicy_snow.retrieval import retrieval_from_parameters, retrieve_snow_depth

# import functions for downloading
from spicy_snow.download.sentinel1 import s1_img_search, hyp3_pipeline, download_hyp3, combine_s1_images
from spicy_snow.download.forest_cover import download_fcf
from spicy_snow.download.snow_cover import download_snow_cover

# import functions for pre-processing
from spicy_snow.processing.s1_preprocessing import merge_partial_s1_images, s1_orbit_averaging,\
s1_clip_outliers, subset_s1_images, ims_water_mask, s1_incidence_angle_masking, merge_s1_subsets

# import the functions for snow_index calculation
from spicy_snow.processing.snow_index import calc_delta_VV, calc_delta_cross_ratio, \
    calc_delta_gamma, clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth

# import the functions for wet snow flag
from spicy_snow.processing.wet_snow import id_newly_frozen_snow, id_newly_wet_snow, \
    id_wet_negative_si, flag_wet_snow

# setup root logger
from spicy_snow.utils.spicy_logging import setup_logging

In [ ]:
work_dir = Path('~/scratch/broken-combo/').expanduser()
os.makedirs(work_dir, exist_ok = True)

dates = get_input_dates('2021-03-16', '2021-03-01')
area = shapely.geometry.box(-117, 44, -116, 45)

out_nc = Path(f'~/scratch/broken-combo/spicy-lowman-v1.nc').expanduser()

search_results = s1_img_search(area, dates)
jobs = hyp3_pipeline(search_results, job_name = 'broken-combo_v4', existing_job_name = 'broken-combo_v4')

In [ ]:
imgs = download_hyp3(jobs, area, outdir = work_dir.joinpath('tmp'), clean = False)

In [ ]:
ds = combine_s1_images(imgs)

In [ ]:
backscatter = np.random.randn(10, 10, 25, 3)

# make some np nan cuts out of backscatter images
for i in range(0, 25, 5):
    backscatter[:3, :, i + 0, :] = np.nan
    backscatter[3:, :, i + 1, :] = np.nan

    backscatter[3:, :, i + 2, :] = np.nan
    backscatter[:3, :, i + 3, :] = np.nan
    backscatter[7:, :, i + 3, :] = np.nan
    backscatter[:7, :, i + 4, :] = np.nan


times = [np.datetime64(t) for t in ['2020-01-01T00:00','2020-01-01T00:10','2020-01-02T10:10', '2020-01-02T10:20', '2020-01-02T10:40']]
times_full = []
[times_full.extend([t + pd.Timedelta(f'{i} days') for t in times]) for i in range(0, 5 * 12, 12)]
orbits = np.tile(np.array([24, 24, 65, 65, 65]), reps = 5)

abs_orbits = [31191, 31191, 28888, 28888, 28888]
abs_full = []
[abs_full.extend([o + i for o in abs_orbits]) for i in range(0, 5 * 12, 12)]
x = np.linspace(0, 9, 10)
y = np.linspace(10, 19, 10)
lon, lat = np.meshgrid(x, y)

test_ds = xr.Dataset(
    data_vars = dict(
        s1 = (["x", "y", "time", "band"], backscatter),
    ),

    coords = dict(
        x = (["x"], x),
        y = (["y"], y),
        band = ['VV', 'VH', 'inc'],
        time = times_full,
        relative_orbit = (["time"], orbits),
        absolute_orbit = (["time"], abs_full)))

test_ds = test_ds.drop_isel(time = 0)
test_ds = test_ds.drop_isel(time = 3)

In [ ]:
# split out each relative orbit and make its own dataset
for orbit_num, orbit_ds in test_ds.groupby('relative_orbit'):
    for absolute_num, abs_ds in orbit_ds.groupby('absolute_orbit'):
        # if only 1 image in this absolute orbit go to the next
        if len(abs_ds.time) == 1:
            continue

        # combine all images into the first image time step
        combo_imgs = abs_ds.mean(dim = 'time')

        # set first image in dataset to this new combined image
        test_ds.loc[{'time' : abs_ds.isel(time = 0).time}] = combo_imgs

        # drop images that have been combined
        test_ds = test_ds.drop_sel(time = abs_ds.isel(time = slice(1, len(abs_ds.time))).time)

In [ ]:
np.sum(np.isnan(test_ds.isel(time = 0).sel(band = 'VV'))) == 70

In [ ]:
ds.isel(time = 0)['s1'].sel(band = 'VV').plot()

In [ ]:
ds.isel(time = slice(1, len(ds.time)))

In [ ]:
res

In [ ]:
ds = merge_partial_s1_images(ds)

In [ ]:
ds.sel(band = "inc").isel(time = 0)['s1'].plot()